In [1]:
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/val
!mkdir dataset/test
!mkdir dataset/annotations
!mkdir dataset/train_labels
!mkdir dataset/val_labels
!mkdir dataset/test_labels


In [2]:
import os
images=os.listdir('/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images')
print(len(images))

14343


In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [3]:
import numpy as np
from tqdm import tqdm
import shutil
from PIL import Image
import cv2
import os
import random


def annotations(mask) :
    ans=[]
    num_labels, labels = cv2.connectedComponents(mask.astype(np.uint8))
    #print(num_labels, labels)
    for i in range(1, num_labels) :
        ys, xs = np.where(labels == i)
        min_x, max_x = xs.min(), xs.max()
        min_y, max_y = ys.min(), ys.max()
        class_id=mask[ys[0], xs[0]]
        #print(class_id, xs[0],ys[0])
        x_center=(min_x+max_x)/2/512
        y_center=(min_y+max_y)/2/512
        width=(max_x-min_x+2)/512
        height=(max_y-min_y+2)/512
        ans.append([class_id, x_center, y_center, width, height])
    return ans

images=os.listdir('/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images')
for i  in tqdm(range(len(images))) :
    CASE_IDENTIFIER=str(i).zfill(5)
    


    
    old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/masks/'+images[i]
    img = Image.open(old_path)
    img = np.array(img, dtype=np.uint8)
    if np.max(img)==2 :
        for j in range(3) :
            if j==1 :
                img1=np.fliplr(img)
            elif j==2 :
                img1=np.flipud(img)
            else :
                img1=img
            anns=annotations(img1)
            new_path='/kaggle/working/dataset/train_labels/'+CASE_IDENTIFIER+'_'+str(j)+'.txt'
            with open(new_path, "w", encoding="utf-8") as f:
                for ann in anns:
                    if ann[0]==2 :
                        f.write(f'{1} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')
        
            old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images/'+images[i]
            image = Image.open(old_path)
            image = np.array(image, dtype=np.uint8)
            if j==1 :
                img1=np.fliplr(image)
            elif j==2 :
                img1=np.flipud(image)
            else :
                img1=image
            new_path='/kaggle/working/dataset/train/'+CASE_IDENTIFIER+'_'+str(j)+'.png'
            img1 = Image.fromarray(img1)
            img1.save(new_path)




    else :
        if np.max(img)==1 :
            anns=annotations(img)
            new_path='/kaggle/working/dataset/train_labels/'+CASE_IDENTIFIER+'.txt'
            with open(new_path, "w", encoding="utf-8") as f:
                for ann in anns:
                    if ann[0]!=1 :
                        f.write(f'{0} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')
        
            old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images/'+images[i]
            new_path='/kaggle/working/dataset/train/'+CASE_IDENTIFIER+'.png'
            shutil.copy(old_path, new_path)
        elif random.random()<0.2 :
            anns=annotations(img)
            new_path='/kaggle/working/dataset/train_labels/'+CASE_IDENTIFIER+'.txt'
            with open(new_path, "w", encoding="utf-8") as f:
                for ann in anns:
                    if ann[0]!=1 :
                        f.write(f'{0} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')
                        
            old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images/'+images[i]
            new_path='/kaggle/working/dataset/train/'+CASE_IDENTIFIER+'.png'
            shutil.copy(old_path, new_path)

100%|██████████| 14343/14343 [06:31<00:00, 36.67it/s]


In [4]:
len(os.listdir('/kaggle/working/dataset/train/'))

7896

In [5]:



images=os.listdir('/kaggle/input/lidc-idri-preprocessed/CT images and masks/val/images')
for i  in tqdm(range(len(images))) :
    CASE_IDENTIFIER=str(i).zfill(5)


    
    old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/val/masks/'+images[i]
    img = Image.open(old_path)
    img = np.array(img, dtype=np.uint8)
    if np.max(img)<3 :
        anns=annotations(img)
        new_path='/kaggle/working/dataset/val_labels/'+CASE_IDENTIFIER+'.txt'
        with open(new_path, "w", encoding="utf-8") as f:
            for ann in anns:
                if ann[0]==2 :
                    f.write(f'{1} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')


        old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/val/images/'+images[i]
        new_path='/kaggle/working/dataset/val/'+CASE_IDENTIFIER+'.png'
        shutil.copy(old_path, new_path)

100%|██████████| 3391/3391 [01:34<00:00, 35.99it/s]


In [7]:

images=os.listdir('/kaggle/input/lidc-idri-preprocessed/CT images and masks/test/images')
for i  in tqdm(range(len(images))) :
    CASE_IDENTIFIER=str(i).zfill(5)


    
    old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/test/masks/'+images[i]
    img = Image.open(old_path)
    img = np.array(img, dtype=np.uint8)
    if np.max(img)<3 :
        anns=annotations(img)
        new_path='/kaggle/working/dataset/test_labels/'+CASE_IDENTIFIER+'.txt'
        with open(new_path, "w", encoding="utf-8") as f:
            for ann in anns:
                if ann[0]==2 :
                    f.write(f'{1} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')

        
        old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/test/images/'+images[i]
        new_path='/kaggle/working/dataset/test/'+CASE_IDENTIFIER+'.png'
        shutil.copy(old_path, new_path)

100%|██████████| 3330/3330 [00:15<00:00, 216.76it/s]


In [21]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as F
from PIL import Image
import numpy as np


In [30]:
class YoloDetectionDataset(Dataset):
    def __init__(self, img_dir, label_dir, transforms=None, img_ext=".png"):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.image_files = sorted([f for f in os.listdir(img_dir) if f.endswith(img_ext)])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace('.png', '.txt'))

        image = Image.open(img_path).convert("RGB")
        w, h = image.size

        boxes = []
        labels = []

        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    # YOLO → COCO
                    xmin = (x_center - width / 2) * w
                    xmax = (x_center + width / 2) * w
                    ymin = (y_center - height / 2) * h
                    ymax = (y_center + height / 2) * h
                    boxes.append([xmin, ymin, xmax, ymax])
                    labels.append(1)  # malignant class

        if boxes:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
        else:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx])
        }

        if self.transforms:
            image = self.transforms(image)

        return image, target


In [31]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = YoloDetectionDataset(
    img_dir="/kaggle/working/dataset/train",
    label_dir="/kaggle/working/dataset/train_labels",
    transforms=transform
)

val_dataset = YoloDetectionDataset(
    img_dir="/kaggle/working/dataset/val",
    label_dir="/kaggle/working/dataset/val_labels",
    transforms=transform
)

test_dataset = YoloDetectionDataset(
    img_dir="/kaggle/working/dataset/test",
    label_dir="/kaggle/working/dataset/test_labels",
    transforms=transform
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


In [25]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 185MB/s]  


In [26]:
import torch
import time
import copy

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Оптимизатор
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


In [28]:
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0
    for images, targets in data_loader:
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        total_loss += losses.item()
    return total_loss / len(data_loader)


In [32]:
num_epochs = 30
best_model_wts = copy.deepcopy(model.state_dict())
best_loss = float("inf")

for epoch in range(num_epochs):
    loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss:.4f}")

    if loss < best_loss:
        best_loss = loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), "best_model.pth")
        print("Saved best model.")



Epoch 1/30, Training Loss: 0.0889
Saved best model.
Epoch 2/30, Training Loss: 0.0611
Saved best model.
Epoch 3/30, Training Loss: 0.0506
Saved best model.


KeyboardInterrupt: 

In [33]:
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [34]:
model.eval()
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [35]:
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [36]:
all_preds = []
all_targets = []

with torch.no_grad():
    for images, targets in test_loader:
        images = [img.to(device) for img in images]
        outputs = model(images)

        for output, target in zip(outputs, targets):
            pred_boxes = output['boxes'].cpu()
            pred_scores = output['scores'].cpu()
            pred_labels = output['labels'].cpu()

            true_boxes = target['boxes'].cpu()
            true_labels = target['labels'].cpu()

            all_preds.append({
                'boxes': pred_boxes,
                'scores': pred_scores,
                'labels': pred_labels,
            })

            all_targets.append({
                'boxes': true_boxes,
                'labels': true_labels,
            })


In [37]:
pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [38]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
metric = MeanAveragePrecision()
metric.update(all_preds, all_targets)
results = metric.compute()
print(results)

{'map': tensor(0.3775), 'map_50': tensor(0.6087), 'map_75': tensor(0.4332), 'map_small': tensor(0.3654), 'map_medium': tensor(0.5666), 'map_large': tensor(-1.), 'mar_1': tensor(0.5262), 'mar_10': tensor(0.5741), 'mar_100': tensor(0.5741), 'mar_small': tensor(0.5637), 'mar_medium': tensor(0.7211), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor(1, dtype=torch.int32)}


In [40]:
import torch

def box_iou(boxes1, boxes2):

    area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])
    area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N, M, 2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N, M, 2]

    wh = (rb - lt).clamp(min=0)  # [N, M, 2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N, M]

    union = area1[:, None] + area2 - inter
    iou = inter / union
    return iou

def calculate_precision_recall(all_preds, all_targets, iou_threshold=0.5, score_threshold=0.5):
    TP, FP, FN = 0, 0, 0

    for preds, targets in zip(all_preds, all_targets):
        pred_boxes = preds['boxes']
        pred_scores = preds['scores']
        pred_boxes = pred_boxes[pred_scores > score_threshold]
        gt_boxes = targets['boxes']

        if len(pred_boxes) == 0:
            FN += len(gt_boxes)
            continue
        if len(gt_boxes) == 0:
            FP += len(pred_boxes)
            continue

        ious = box_iou(pred_boxes, gt_boxes)
        matched_gt = set()

        for i in range(len(pred_boxes)):
            max_iou, idx = torch.max(ious[i], dim=0)
            if max_iou >= iou_threshold and idx.item() not in matched_gt:
                TP += 1
                matched_gt.add(idx.item())
            else:
                FP += 1

        FN += len(gt_boxes) - len(matched_gt)

    precision = TP / (TP + FP + 1e-6)
    recall = TP / (TP + FN + 1e-6)

    return precision, recall


In [41]:
precision, recall = calculate_precision_recall(all_preds, all_targets, iou_threshold=0.5, score_threshold=0.5)
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}')

Precision: 0.1475, Recall: 0.8042


In [43]:
precision, recall = calculate_precision_recall(all_preds, all_targets, iou_threshold=0.5, score_threshold=0.9)
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}')

Precision: 0.3883, Recall: 0.7413


In [44]:
precision, recall = calculate_precision_recall(all_preds, all_targets, iou_threshold=0.5, score_threshold=0.95)
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}')

Precision: 0.5172, Recall: 0.6853


In [45]:
precision, recall = calculate_precision_recall(all_preds, all_targets, iou_threshold=0.5, score_threshold=0.97)
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}')

Precision: 0.6290, Recall: 0.6224


In [46]:
precision, recall = calculate_precision_recall(all_preds, all_targets, iou_threshold=0.1, score_threshold=0.97)
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}')

Precision: 0.6360, Recall: 0.6294
